In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import glob
import sys
import os
from pathlib import Path

import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../src')
from clf_parser import CLFParse

In [3]:
# For modin
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Modify path here for your particular system
data_dir = Path('/Users/kaipak/datasets/pds/pds_logs/report_service/logs/final/rings/rings-apache-metrics')
tools_logs = sorted(glob.glob(str(data_dir) + "/tools_*"))
tools2_logs = sorted(glob.glob(str(data_dir) + "/tools2*"))
server1_logs = sorted(glob.glob(str(data_dir) + "/server1*"))
server2_logs = sorted(glob.glob(str(data_dir) + "/server2*"))
#server2_logs = [f'{str(data_dir)}/server2-pds-rings-apache2.2019-02-04.log']

In [5]:
len(tools_logs)

201

In [6]:
len(tools2_logs)

647

In [7]:
len(server1_logs)

447

In [8]:
len(server2_logs)

1240

In [9]:
parser_tools = CLFParse(tools_logs)
parser_tools2 = CLFParse(tools2_logs)
parser_server1 = CLFParse(server1_logs)
parser_server2 = CLFParse(server2_logs)

In [10]:
%%time
parser_tools.parse_files()
parser_tools.df_logs['log_name'] = 'tools'

Logfiles list contains 201 files.


100%|██████████| 201/201 [00:00<00:00, 585.83it/s]


Expanding datetime field...
0.09m to process log files.
CPU times: user 1.03 s, sys: 209 ms, total: 1.24 s
Wall time: 5.43 s


In [12]:
parser_tools.df_logs.sample(5)

,ip,identd,userid,datetime,request,status,size,referer,user_agent,date,month_year,DOW,hour,log_name
1314,2.152.103.57,None,None,2021-11-27 16:31:05+00:00,GET /opus/__api/image/thumb/co-iss-n1363912382...,200,747,https://opus.pds-rings.seti.org/opus/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:9...,2021-11-27,11-2021,Saturday,16,tools
104,208.115.199.22,None,None,2021-11-28 09:56:06+00:00,HEAD / HTTP/1.1,200,5083,https://tools.pds-rings.seti.org,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,2021-11-28,11-2021,Sunday,9,tools
52,23.250.19.242,None,None,2022-02-09 23:52:15+00:00,\n,400,5240,None,None,2022-02-09,02-2022,Wednesday,23,tools
2049,208.115.199.22,None,None,2021-11-27 04:21:17+00:00,HEAD / HTTP/1.1,200,5083,https://opus.pds-rings.seti.org,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,2021-11-27,11-2021,Saturday,4,tools
1568,107.126.40.131,None,None,2021-11-30 18:09:24+00:00,GET /downloads/pdsrms-2021-11-30T10-09-21-0636...,200,3197821,https://opus.pds-rings.seti.org/opus/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2021-11-30,11-2021,Tuesday,18,tools


In [13]:
%%time
# This might take a while. 
parser_tools2.parse_files()
parser_tools2.df_logs['log_name'] = 'tools2'

Logfiles list contains 647 files.


100%|██████████| 647/647 [00:12<00:00, 51.11it/s]


Expanding datetime field...
1.42m to process log files.
CPU times: user 58.1 s, sys: 4.86 s, total: 1min 2s
Wall time: 1min 25s


In [ ]:
parser_tools2.df_logs.shape

In [ ]:
parser_tools2.df_logs.sample(5)

In [ ]:
parser_tools2.to_file(type="feather", filepath="../datasets/rings_tools2.feather")

In [ ]:
%%time
# This might take a while. 
parser_server1.parse_files()
parser_server1.df_logs['log_name'] = 'server1'

In [ ]:
parser_server1.df_logs.shape

In [ ]:
%%time
# This might take a while. 
parser_server2.parse_files()
parser_server2.df_logs['log_name'] = 'server2'

In [ ]:
parser_server2.df_logs

In [ ]:
parser_server1.to_file(type="feather", filepath="../datasets/rings_server1.feather")

## Example Read in File and Create Data Viz


In [ ]:
%%time
df_server1 = pd.read_parquet('../datasets/rings_server1.pq')

In [ ]:
df_server1.shape

In [ ]:
df_server1.head()

In [ ]:
df_summary = (
    df_server1.
    groupby(by='status').
    size().to_frame('requests').reset_index().
    sort_values(by='requests', ascending=False)
)
fig, ax = plt.subplots(figsize=(15, 12))
bar_p = sns.barplot(data=df_summary, x='status', y='requests', 
                    order=df_summary.sort_values('requests', ascending=False).status)

In [ ]:
df_summary

In [ ]:
df_server1.shape

In [ ]:
df = pd.concat([parser_tools1.df_logs, parser_tools2.df_logs, parser_server1.df_logs, parser_server2.df_logs])

In [ ]:
df.shape

In [ ]:
parser_server2.df_logs.shape

In [ ]:
df.reset_index(drop=True).to_feather('../datasets/tools1_tools2_server1.feather')

In [ ]:
%%time
df = pd.read_feather('../datasets/tools1_tools2_server1.feather')

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.sort_values(by='date', inplace=True)

In [ ]:
df.head()

In [ ]:
df_sum = (
    df.groupby(by=['month_year', 'date']).agg({'size': 'sum', 'ip': 'count'}).
    reset_index(drop=False).rename(columns={'size': 'data_transmit (GB)', 'ip': 'requests'})
)

# Basic Transformations
df_sum['data_transmit (GB)'] = df_sum['data_transmit (GB)'] / (8 * 10**9)
df_sum['date'] = pd.to_datetime(df_sum['date'])
df_sum['data_transmit_range'] = pd.cut(df_sum['data_transmit (GB)'], 
                                       bins=[0, .1, 1, 10, 100, 200, 500, 1000, 100000],
                                       labels=['0-100MB', '100MB-1GB', '1-10GB', '10-100GB', '100-200GB', '200-500GB', 
                                               '500GB-1TB', '>1TB'])
print(df_sum.dtypes)

In [ ]:
import plotly.express as px
df_sum.sort_values(by='date', inplace=True)
fig = px.line(df_sum, x="date", y="data_transmit (GB)", log_y=True)
fig.show()